In [37]:
import nltk
#nltk.download('reuters')
from nltk.corpus import reuters
print('training files : ', len([fid for fid in reuters.fileids() if fid[:5] == 'train'])) #les 5 premières lettre
print('testing files : ', len([fid for fid in reuters.fileids() if fid[:4] == 'test']))
print('total files : ', len(reuters.fileids()))


training files :  7769
testing files :  3019
total files :  10788


In [63]:
import nltk
from nltk.corpus import reuters
import pandas as pd

def join_tokens(tokens):
    return ' '.join(tokens)

data = []

for file_id in reuters.fileids():
    text = join_tokens(reuters.words(file_id))
    category = 'grain' if 'grain' in reuters.categories(file_id) else 'non-grain'
    label = 1 if category == 'grain' else 0
    data.append((file_id, text, label))

df = pd.DataFrame(data, columns=['file_id', 'text', 'category'])

train_ids = [file_id for file_id in reuters.fileids() if file_id.startswith('training/')]
test_ids = [file_id for file_id in reuters.fileids() if file_id.startswith('test/')]

train_df = df[df['file_id'].isin(train_ids)]
test_df = df[df['file_id'].isin(test_ids)]

train_texts_df = train_df.drop(['category', 'file_id'], axis=1)
train_labels_df = train_df['category']
train_labels_df = pd.DataFrame(train_labels_df, columns=['category'])

test_texts_df = test_df.drop(['category', 'file_id'], axis=1)
test_labels_df = test_df['category']
test_labels_df = pd.DataFrame(test_labels_df, columns=['category'])

print(train_texts_df.head())
print(train_labels_df.head())

print(test_texts_df.head())
print(test_labels_df.head())


                                                   text
3019  BAHIA COCOA REVIEW Showers continued throughou...
3020  COMPUTER TERMINAL SYSTEMS & lt ; CPML > COMPLE...
3021  N . Z . TRADING BANK DEPOSIT GROWTH RISES SLIG...
3022  NATIONAL AMUSEMENTS AGAIN UPS VIACOM & lt ; VI...
3023  ROGERS & lt ; ROG > SEES 1ST QTR NET UP SIGNIF...
      category
3019         0
3020         0
3021         0
3022         0
3023         0
                                                text
0  ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPA...
1  CHINA DAILY SAYS VERMIN EAT 7 - 12 PCT GRAIN S...
2  JAPAN TO REVISE LONG - TERM ENERGY DEMAND DOWN...
3  THAI TRADE DEFICIT WIDENS IN FIRST QUARTER Tha...
4  INDONESIA SEES CPO PRICE RISING SHARPLY Indone...
   category
0         0
1         1
2         0
3         1
4         0


In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import ComplementNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer

import numpy as np
import pandas as pd
from sklearn.datasets import fetch_rcv1
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


pipeline1 = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', ComplementNB())])

pipeline2 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', SGDClassifier(loss='hinge')),])

pipeline3 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge')),])

pipeline4 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', ComplementNB()),])


In [77]:
parameter_grid1 = {
    'vect__max_df': [0.75],
    'vect__ngram_range': [(1,1), (1,2)],
    'vect__use_idf': [True, False],
    'vect__norm': ['l1', 'l2'],
    'clf__alpha': [10.0] 
}

parameter_grid2 = {
    'tfidf__max_df': [0.2, 0.5],
    'tfidf__use_idf': [True, False],
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'clf__penalty': ['l1', 'l2'],
    'clf__alpha': [0.01]
}

parameter_grid3 = {
    'vect__max_df': [0.2, 0.5],
    'vect__ngram_range': [(1, 1), (1, 2)],
    'tfid__use_idf': [True, False],
    'tfid__norm': ['l1', 'l2'],
    'clf__alpha': [0.0001, 0.001, 0.001],
    'clf__max_iter': [1000, 2000]
}

parameter_grid4 = {
    'vect__max_df': [0.2, 0.5],
    'vect__ngram_range': [(1, 1), (1, 2)],
    'tfid__use_idf': [True, False],
    'tfid__norm': ['l1', 'l2'],
    'clf__alpha': [0.001, 0.001],
}

scorer = make_scorer(f1_score, pos_label=1)
#essayer avec moins de folds
grid_search1 = GridSearchCV(pipeline1, parameter_grid1, scoring='f1', cv=5, n_jobs=-1, verbose=1)
grid_search1.fit(train_texts_df['text'], train_labels_df['category'])

grid_search2 = GridSearchCV(pipeline2, parameter_grid2, scoring='f1', cv=5, n_jobs=-1, verbose=1)
grid_search2.fit(train_texts_df['text'], train_labels_df['category'])
#
grid_search3 = GridSearchCV(pipeline3, parameter_grid3, scoring='f1', cv=5, n_jobs=-1, verbose=1)
grid_search3.fit(train_texts_df['text'], train_labels_df['category'])
#
grid_search4 = GridSearchCV(pipeline4, parameter_grid4, scoring='f1', cv=5, n_jobs=-1, verbose=1)
grid_search4.fit(train_texts_df['text'], train_labels_df['category'])

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 96 candidates, totalling 480 fits


ValueError: Invalid parameter 'tfid' for estimator Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier())]). Valid parameters are: ['memory', 'steps', 'verbose'].

In [ ]:

# attention à fitter un modèle sur le test set pour obtenir le f1 score final !
# extraire les best_hyperparameters
result1_df = pd.DataFrame.from_dict(grid_search1.best_params_, orient='index', columns=['Valeur'])
result2_df = pd.DataFrame.from_dict(grid_search2.best_params_, orient='index', columns=['Valeur'])
result3_df = pd.DataFrame.from_dict(grid_search3.best_params_, orient='index', columns=['Valeur'])
result4_df = pd.DataFrame.from_dict(grid_search4.best_params_, orient='index', columns=['Valeur'])

pred1 = grid_search1.predict(test_texts_df['text'])
pred2 = grid_search2.predict(test_texts_df['text'])
pred3 = grid_search3.predict(test_texts_df['text'])
pred4 = grid_search4.predict(test_texts_df['text'])

# afficher les scores avec les paramètres
test_f1_score1 = f1_score(test_labels_df, pred1, pos_label=1)
print("Hyper param pour 1 pipeline-> {result1_df}:.3f")
print("f1 score -> {test_f1_score1}:.3f")

test_f1_score2 = f1_score(test_labels_df, pred2, pos_label=1)
print("Hyper param pour 2 pipeline-> {result2_df}:.3f")
print("f1 score -> {test_f1_score2}:.3f")

test_f1_score3 = f1_score(test_labels_df, pred3, pos_label=1)
print("Hyper param pour 3 pipeline-> {result3_df}:.3f")
print("f1 score -> {test_f1_score3}:.3f")

test_f1_score4 = f1_score(test_labels_df, pred4, pos_label=1)
print("Hyper param pour 4 pipeline-> {result4_df}:.3f")
print("f1 score -> {test_f1_score4}:.3f")

###
